In [1]:
from IPython.display import display
from vnstock import *
import pandas as pd
from datetime import *
from openpyxl import *
import os
from openpyxl.styles import *
from openpyxl.formatting.rule import *
from openpyxl.utils.dataframe import *
from openpyxl.utils import get_column_letter
from openpyxl.utils import column_index_from_string
from stock_processed import stock_ratio_calculation
from ratio import ratio_calculation
from visualization import plot_stock_prices
import shutil

Non-200 response fetching content: 400


In [9]:

co_phieu = 'VRE'
filename_processed = stock_ratio_calculation(co_phieu)

# nhập dataframe ratio và lưu vào file excel tổng đã xử lý
df_ratio, file_name_ratio = ratio_calculation(co_phieu) 
with pd.ExcelWriter(filename_processed, engine='openpyxl', mode='a') as writer:
    df_ratio.to_excel(writer, sheet_name='Tỉ số tài chính', index=False)

# mở bằng openpyxl để căn chỉnh và tô màu

wb = load_workbook(filename_processed)


header_font = Font(bold=True, size=13, name="Times New Roman")
header_fill = PatternFill(fgColor="9aaab5", fill_type="solid")
header_alignment = Alignment(horizontal="center", vertical="center")

# căn chỉnh cho file lịch sử giá:
if 'Giá lịch sử' in wb.sheetnames:
    ws_price = wb['Giá lịch sử']

    for cell in ws_price[1]: # format header
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    
    for cell in ws_price['A'][1:]:
        cell.number_format = "DD/MM/YYYY"
        cell.alignment = Alignment( horizontal = 'center') # format cột ngày cho chuẩn
        ws_price.column_dimensions['A'].width = 15

    for col in ['B','C','D','E','F']:
        for cell in ws_price[col][1:]:
            cell.alignment = Alignment( horizontal = 'center')
    for cell in ws_price['F']:
        cell.number_format = '#,##0' 
    
    for column in ws_price.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column: 
            try:
                if len(str(cell.value)) > max_length :
                    max_length = len(str(cell.value))
            except:
                pass
        ad_width = min(max_length + 2, 40)
    ws_price.column_dimensions[column_letter].width = ad_width


# căn chỉnh báo cáo tài chính
bold_row = [
    'Doanh thu bán hàng và cung cấp dịch vụ',
    'Doanh thu thuần về bán hàng và cung cấp dịch vụ',
    'Lợi nhuận gộp về bán hàng và cung cấp dịch vụ',
    'Lợi nhuận thuần từ hoạt động kinh doanh',
    'Lợi nhuận khác',
    'Tổng lợi nhuận kế toán trước thuế',
    'Lợi nhuận sau thuế thu nhập doanh nghiệp'
]
font_bold = Font(bold=True)
if 'Báo cáo kết quả kinh doanh' in wb.sheetnames:
    ws_key= wb['Báo cáo kết quả kinh doanh']
    #bold đậm header
    for cell in ws_key[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    # bold đậm các hàng cần bold
    for row in ws_key.iter_rows(min_row = 2):
        for cell in row:
            cell.font = Font(size=13, name="Times New Roman") # xét từng row kể từ row thứ 2
            chi_tieu_cell = row[0] # đặt điều kiện để xét

        if chi_tieu_cell.value in bold_row:
            for cell in row:
                cell.font = Font(bold=True,size=13, name="Times New Roman")
    # chỉnh cho các cộ dòng đầu về bên trái
    for cell in ws_key['A'][1:]:
        cell.alignment = Alignment(horizontal="left", vertical="center")
    
    # căn chỉnh cho tất cả các ô tự động aligntment của cột A
    for cell in ws_key['A']: 
        if len(str(cell.value)) > max_length :
            max_length = len(str(cell.value))           
        ad_width = max_length + 10
    ws_key.column_dimensions['A'].width = ad_width

# đúng định dạng số
    for col in ws_key.iter_cols(min_col = 2):
        for cell in col[1:]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            if cell.value != 0:
                cell.number_format = '#,##0;[Red](#,##0)'
            else:
                cell.value = 'N/A'
               

    #thêm left border
    new_line = Side(style = 'thin', color ='000000')
    for row in ws_key.iter_rows():
        for cell in row:
            # tạo ra copy của border gốc đã
            cell.border = Border(right = new_line)
    


#căn chỉnh bảng cân đối kế toán
if 'Bảng cân đối kế toán' in wb.sheetnames:
    ws_balance_sheet = wb['Bảng cân đối kế toán']

    header_font = Font(bold=True, size=13, name="Times New Roman")
    header_fill = PatternFill(fgColor="9aaab5", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")

    # căn chỉnh tiêu đề cho chuẩn
    for cell in ws_balance_sheet[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    #căn chỉnh các cell cho chuẩn
    for column in ws_balance_sheet.iter_cols(min_col=2):
        for cell in column[1:]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.font = Font(name="Times New Roman", size=13)
            if cell.value not in [None, '', 0]:
                cell.number_format = '#,##0;[Red](#,##0)'
            else:
                cell.number_format = 'General'
                cell.value = 'N/A'

    # tự động căn chỉnh độ rộng của các cell hàng 1
    max_length = 0
    for cell in ws_balance_sheet['A'][1:]:
        cell.alignment = Alignment(horizontal="left", vertical="center")
        if len(str(cell.value)) > max_length:
            max_length = len(str(cell.value))
        length = max_length + 3
        ws_balance_sheet.column_dimensions['A'].width = length
        cell.font = Font(name="Times New Roman", size=13)

    # bôi đậm 1 số hàng cần thiết
    bold_values = '''
    A. Tài sản lưu động và đầu tư ngắn hạn
    I. Tiền và các khoản tương đương tiền
    II. Các khoản đầu tư tài chính ngắn hạn
    III. Các khoản phải thu ngắn hạn
    IV. Tổng hàng tồn kho
    V. Tài sản ngắn hạn khác
    B. Tài sản cố định và đầu tư dài hạn
    I. Các khoản phải thu dài hạn
    II. Tài sản cố định
    III. Bất động sản đầu tư
    IV. Tài sản dở dang dài hạn
    V. Các khoản đầu tư tài chính dài hạn
    VI. Tổng tài sản dài hạn khác
    VII. Lợi thế thương mại
    TỔNG CỘNG TÀI SẢN
    NGUỒN VỐN
    A. Nợ phải trả
    I. Nợ ngắn hạn
    II. Nợ dài hạn
    B. Nguồn vốn chủ sở hữu
    I. Vốn chủ sở hữu
    II. Nguồn kinh phí và quỹ khác
    TỔNG CỘNG NGUỒN VỐN
    '''
    bold_list_raw = bold_values.strip().splitlines()
    bold_list_processed = []
    for item in bold_list_raw:
        item_sach = item.strip()
        bold_list_processed.append(item_sach)

    for row in ws_balance_sheet.iter_rows(min_row=2):
        if row[0].value in bold_list_processed:
            for cell in row:
                cell.font = header_font

    #thêm border cho mỗi ô vào bên phải
    new_line = Side(style = 'thin', color ='000000')
    for row in ws_balance_sheet.iter_rows():
        for cell in row:
            cell.border = Border(right = new_line) 

#căn chỉnh cash flow statement
if 'Báo cáo lưu chuyển tiền tệ' in wb.sheetnames:
    ws_cashflow = wb['Báo cáo lưu chuyển tiền tệ']

    header_font = Font(bold=True, size=13, name="Times New Roman")
    header_fill = PatternFill(fgColor="9aaab5", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")

    # căn chỉnh tiêu đề cho chuẩn
    for cell in ws_cashflow[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    #căn chỉnh các cell cho chuẩn
    for column in ws_cashflow.iter_cols(min_col=2):
        for cell in column[1:]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.font = Font(name="Times New Roman", size=13)
            if cell.value not in [None, '', 0]:
                cell.number_format = '#,##0;[Red](#,##0)'
            else:
                cell.number_format = 'General'
                cell.value = 'N/A'
    
    # tự động căn chỉnh độ rộng của các cell hàng 1
    max_length = 0
    for cell in ws_cashflow['A'][1:]:
        cell.alignment = Alignment(horizontal="left", vertical="center")
        if len(str(cell.value)) > max_length:
            max_length = len(str(cell.value))
        length = max_length + 5
        ws_cashflow.column_dimensions['A'].width = length
        cell.font = Font(name="Times New Roman", size=13)
    # bôi đậm 1 số hàng cần thiết
    bold_cf = '''
    I. Lưu chuyển tiền từ hoạt động kinh doanh
    1. Lợi nhuận trước thuế
    2. Điều chỉnh cho các khoản
    3. Lợi nhuận từ hoạt động kinh doanh trước thay đổi vốn lưu động
    Lưu chuyển tiền thuần từ hoạt động kinh doanh
    II. Lưu chuyển tiền từ hoạt động đầu tư
    Lưu chuyển tiền thuần từ hoạt động đầu tư
    III. Lưu chuyển tiền từ hoạt động tài chính
    Lưu chuyển tiền thuần từ hoạt động tài chính
    Lưu chuyển tiền thuần trong kỳ
    Tiền và tương đương tiền đầu kỳ
    Tiền và tương đương tiền cuối kỳ
    '''
    bold_list_raw_cf = bold_cf.strip().splitlines()
    bold_list_processed = [item.strip() for item in bold_list_raw_cf]
    for row in ws_cashflow.iter_rows(min_row=2):
        if row[0].value in bold_list_processed:
            for cell in row:
                cell.font = header_font
    
    #thêm border cho mỗi ô vào bên phải
    new_line = Side(style = 'thin', color ='000000')
    for row in ws_cashflow.iter_rows():
        for cell in row:
            cell.border = Border(right = new_line)

# căn chỉnh sheet tỉ số tài chính
if 'Tỉ số tài chính' in wb.sheetnames:
    ws_ratio = wb['Tỉ số tài chính']

    header_font = Font(bold=True, size=13, name="Times New Roman")
    header_fill = PatternFill(fgColor="9aaab5", fill_type="solid")
    header_alignment = Alignment(horizontal="center", vertical="center")

    # căn chỉnh tiêu đề cho chuẩn
    for cell in ws_ratio[1]:
        cell.font = header_font
        cell.fill = header_fill
        cell.alignment = header_alignment
    # #căn chỉnh các cell cho chuẩn
    for column in ws_ratio.iter_cols(min_col=2):
        for cell in column[1:]:
            cell.alignment = Alignment(horizontal="center", vertical="center")
            cell.font = Font(name="Times New Roman", size=13)
            if isinstance(cell.value, (int, float)):
                cell.number_format = '#,##0.00;[Red](#,##0.00)'
            else:
                cell.value = None  # Để ô rỗng hoàn toàn
    
    # tự động căn chỉnh độ rộng của các cell hàng 1
    max_length = 0
    for cell in ws_ratio['A'][1:]:
        cell.alignment = Alignment(horizontal="left", vertical="center")
        if len(str(cell.value)) > max_length:
            max_length = len(str(cell.value))
        length = max_length + 5
        ws_ratio.column_dimensions['A'].width = length
        cell.font = Font(name="Times New Roman", size=13)
    #thêm border cho mỗi ô vào bên phải
    new_line = Side(style = 'thin', color ='000000')
    for row in ws_ratio.iter_rows():
        for cell in row:
            cell.border = Border(right = new_line)
    
    #bold các hàng cần bold
    bold_ratio = [
        'Nhóm chỉ số Định giá',
        'Nhóm chỉ số Sinh lợi',
        'Nhóm chỉ số Tăng trưởng',
        'Nhóm chỉ số Thanh khoản',
        'Nhóm chỉ số Hiệu quả hoạt động',
        'Nhóm chỉ số Đòn bẩy tài chính',
        'Nhóm chỉ số Dòng tiền' ]
    for row in ws_ratio.iter_rows(min_row=2):
        if row[0].value in bold_ratio:
            for cell in row:
                cell.font = header_font
    #vẽ chart____
    chart= plot_stock_prices(co_phieu)
    img_pe, img_growth, img_roe, img_netprofit_cf = chart
    # xoá ảnh cũ trong excel nếu có
    if hasattr(ws_ratio, '_images'):
        ws_ratio._images.clear()
    ws_ratio.add_image(img_pe, 'J2')
    ws_ratio.add_image(img_growth, 'J20')
    ws_ratio.add_image(img_roe, 'J38')
    ws_ratio.add_image(img_netprofit_cf, 'J56')

#tạo file đẹp từ openpyxl
if not os.path.exists('data_đẹp'):
    os.makedirs('data_đẹp')
today = date.today().strftime('%d-%m-%Y')
filename_openpyxl = f'data_đẹp/{co_phieu}_{today}_financials_openpyxl.xlsx'

wb.save(filename_openpyxl)


os.remove('pe_ratio_plot.png')
os.remove('growth_revenue_profit_plot.png')
os.remove('roe_roa_roce_plot.png')
os.remove('net_profit_cf_margin_plot.png')
if os.path.exists('data_processed'):
    shutil.rmtree('data_processed')

# Xóa toàn bộ thư mục data_raw nếu tồn tại
if os.path.exists('data_raw'):
    shutil.rmtree('data_raw')







A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
Traceback (most recent call last):
  File "/Users/an/Desktop/suici_mouse_scraper/đã xong/stock_excel_automation/stock_processed.py", line 34, in stock_ratio_calculation
    with pd.ExcelWriter(output_filename, engine='openpyxl') as writer:
         ~~~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/an/Desktop/suici_mouse_scraper/my_python_env/lib/python3.13/site-packages/pandas/io/excel/_openpyxl.py", line 61, in __init__
    super().__init__(
    ~~~~~~~~~~~~~~~~^
        path,
        ^^^^^
    ...<3 lines>...
        engine_kwargs=engine_kwargs,
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/Users/an/Desktop/suici_mouse_scraper/my_p

Lỗi khi tạo file excel tổng hợp: Cannot save file into a non-existent directory: 'data_processed'



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
'Y' is deprecated and will be removed in a future version, please use 'YE' instead.

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
DataFrame.applymap has been deprecated. Use DataFrame.map instead.
'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
